# On homomorphisms

This section is devoted to understand what are homomoprhism and how can we compute on the ciphertext sums and multiplications. At the end of it we will have an idea on what is fully homomorphic encryption.

## Homomorphisms in algebraic groups

Let's put some formalism first. Let $(G, \bigstar)$ and $(H, \square)$ be algebraic groups. A group homomorphism from $G$ to $H$ is a function

$$f: G \rightarrow H$$

such that

$$f(x \bigstar y) = f(x)\square f(y)$$
$$\forall x, y \in A$$

As a simple example, take the two groups $(\mathbb{R}, +)$ and $(\mathbb{R}^+, \cdot)$, i.e. the real numbers with addition group and real positive numbers with multiplication group. Take the function $f(x)=exp(x)$, then $f$ defines an homomorphism with the two groups since

$$f(x+y)=e^{x+y}=e^x\cdot e^y=f(x)\cdot f(y)$$

### Paillier 

Paillier cryptosystem defines an homomorphism on the group $(\mathbb{Z}_{N^2}, +)$ where $f$ is the encryption function, this is:

$$Enc: (\mathbb{Z}_{N}, +) \rightarrow (\mathbb{Z}_{N^2}, \cdot)$$

where the function $Enc(x)$ is

$$Enc(x)=g^x \cdot r^N \pmod{N^2}$$

and $N$ is the product of two prime numbers $p$ and $q$ such that the maximum common multiple of $N$ and $(p-1)(q-1)$ is 1. At the same time, $g$ is a random element of $\mathbb{Z}_{N^2}$ and $r$ is a random number the range $0<r<N$ and part of the group ($\mathbb{Z}_N, \cdot$) i.e. $gcm(r, N)=1$.

There is another calculated parameter kept private 

$$\mu=(L(g^{\lambda}\pmod{N^2}))^{-1}\pmod{N}$$

where the function $L$ is defined as

$$L(x)=\frac{x-1}{n}$$

taking the integer part of the divison and $\lambda$ is the least common multiple of the order of the primes i.e. of $p-1$ and $q-1$. 

One can decrypt the ciphertext $c$ and recover the original message $x$ as

$$x=L(c^{\lambda} \pmod{N^2})\cdot \mu \pmod{N}$$

$$Enc(m_1+m_2)=Enc(m_1)\cdot Enc(m_2)$$

Thus the encryption of two plaintexts in $N^2$ is the multipication of their ciphertexts. Let's try to show this in some examples, the proof of the homomorphic encryption can be found [elsewhere](https://doi.org/10.1016/B978-0-12-801595-7.00005-7).

In [1]:
from crypt import PaillierKeyGenerator, PaillierEncrypt, PaillierDecrypt
from random import randrange, seed

In [160]:
bits = 16
seed(3)

PublicKeyPaillier, PrivateKeyPaillier = PaillierKeyGenerator(bits)
N = PublicKeyPaillier[0]
g = PublicKeyPaillier[1]

l = PrivateKeyPaillier[1]
mu = PrivateKeyPaillier[2]

m1, m2 = randrange(2, 2**(bits//4)), randrange(2, 2**(bits//4))
print(f"Paillier parameters:\n-Public:\n\tN={N}\n\tg={g}")
print(f"\n-Private:\n\tN={N}\n\tlambda={l}\n\tmu={mu}")
print(f"Plaintext messages:\nm1={m1}\nm2={m2}\n")

Paillier parameters:
-Public:
	N=3863609707
	g=6949833295068056498

-Private:
	N=3863609707
	lambda=1931742680
	mu=305714079
Plaintext messages:
m1=9
m2=15



We will test that 

$$m_1+m_2 \pmod{N} = Dec(Enc(m_1)\cdot Enc(m_2)\pmod{N^2})$$

on plaintext because the encryption on Paillier is not bijective, i.e. applying the same algorithm on the same input we won't get the same result, that is because it depends on a randomly sampled $r$ (look at the source code of encryption). 

In [161]:
print(f"Sum modulo N of plaintexts: {(m1+m2)%N}")
prod_ciphertext = PaillierEncrypt(m1, PublicKeyPaillier)*PaillierEncrypt(m2, PublicKeyPaillier)%(N**2)
decripted_prod = PaillierDecrypt(prod_ciphertext, PrivateKeyPaillier)
print(f"Decription of the product encoded {decripted_prod}")

Sum modulo N of plaintexts: 24
Decription of the product encoded 24


It can also be shown that

$$Dec(Enc(m1)g^{m_2}\pmod{N^2})=m_1+m_2 \pmod{N}$$

In [163]:
from crypt import fastPowering
PaillierDecrypt(PaillierEncrypt(m1, PublicKeyPaillier)*fastPowering(g, m2, N**2), PrivateKeyPaillier)

24

but this does not define any homomorphism since we are pultiplying an exponenciated plaintext (from the first group) to a ciphertext (from the second group). 

### El Gamal

El Gamal cryptosystem defines a homomorphism on $(\mathbb{Z}_p, \cdot)$ 

$$Enc: (\mathbb{Z}_p, \cdot) \rightarrow (\mathbb{Z}_p, \cdot) \times (\mathbb{Z}_p, \cdot)$$

s. t 

$$Enc(m) = c = (c_1, c_2)$$

This cryptosystem has been explained in another notebook so I refer the reader there to understand it better. The muliplication in the plaintext is defined the usual way but on the ciphertext we multiply pointwise (Shorr):

$$Enc(m_x) = (c_{x1}, c_{x2})$$
$$Enc(m_y) = (c_{y1}, c_{y2})$$
$$Enc(m_x)\cdot Enc(m_y) = (c_{x1}\cdot c_{y1}, c_{x2}\cdot c_{y2})$$

In [175]:
from crypt import ElGamalKeyGenerator, ElGamalEncrypt, ElGamalDecrypt

PublicKeyElGamal, PrivateKeyElGamal = ElGamalKeyGenerator(bits)
A = PublicKeyElGamal[0]
g = PublicKeyElGamal[1]
p = PublicKeyElGamal[2]

sk = PrivateKeyElGamal[0]

print(f"ElGamal parameters:\n-Public:\n\tA={A}\n\tg={g}\n\tp={p}")
print(f"\n-Private:\n\tsk={sk}\n\tp={p}\n")
print(f"Plaintext messages:\nm1={m1}\nm2={m2}\n")

ElGamal parameters:
-Public:
	A=1799
	g=18202
	p=56131

-Private:
	sk=25791
	p=56131

Plaintext messages:
m1=9
m2=15



We have to test the homomorphism:

$$Enc(m_1\cdot m_2)=Enc(m_1)\cdot Enc(m_2) = (c_{x1}\cdot c_{y1}, c_{x2} \cdot c_{y2})$$

or equivalently 

$$Dec(Enc(m_1)\cdot Enc(m_2))=m_1\cdot m_2$$

In [195]:
c1 = ElGamalEncrypt(m1, PublicKeyElGamal)
c2 = ElGamalEncrypt(m2, PublicKeyElGamal)

# Plaintext multiplication
m1m2 = m1*m2%p

#encryption and decryption of the multiplied
#m1m2p = ElGamalDecrypt(ElGamalEncrypt(m1m2, PublicKeyElGamal),PrivateKeyElGamal)

cipher_mult = (c1[0]*c2[0]%p, c1[1]*c2[1]%p)
dec_cipher_mult = ElGamalDecrypt(cipher_mult, PrivateKeyElGamal)
assert m1m2==dec_cipher_mult, "something went wrong"


print(f"ciphertext 1 = {c1}\nciphertext 2 = {c2}\n")
print(f"Product of m1 and m2 in ciphertext = {cipher_mult}")
print(f"Decrypting the product of ciphertext {dec_cipher_mult}")
print(f"Product of the plaintexts {m1m2}")

ciphertext 1 = (16859, 2894)
ciphertext 2 = (23824, 7856)

Product of m1 and m2 in ciphertext = (31511, 2209)
Decrypting the product of ciphertext 143
Product of the plaintexts 143


Wehey! the encription function defined in ElGamal cryptosystem is homomorphic with respect to the product modulo $p$.

### RSA



In [196]:
from crypt import RSAKeyGenerator, RSAEncrypt, RSADecrypt

PublicKeyRSA, PrivateKeyRSA = RSAKeyGenerator(bits)

N = PublicKeyRSA[0]
e = PublicKeyRSA[1]
d = PrivateKeyRSA[1]

m1, m2 = randrange(2, 2**(bits//4)), randrange(2, 2**(bits//4))

print(f"RSA parameters:\n-Public:\n\tN={N}\n\te={e}")
print(f"\n-Private:\n\td={d}")
print(f"Plaintext messages:\nm1={m1}\nm2={m2}\n")

RSA parameters:
-Public:
	N=2734601263
	e=106019107

-Private:
	d=38917579
Plaintext messages:
m1=14
m2=5



In [198]:
mult = m1*m2%N
mult2 = RSADecrypt(RSAEncrypt(m1, PublicKeyRSA)*RSAEncrypt(m2, PublicKeyRSA)%N, PrivateKeyRSA)

assert mult==mult2, "something went wrong"


print(f"plaintext multiplication: {mult}")
print(f"ciphertext multiplication and decryption: {mult2}")

plaintext multiplication: 70
ciphertext multiplication and decryption: 70


Up until now we have seen that RSA, ElGamal and Paillier encryption schemes are homomorphic with a certain operation. Furthermore this operation can be applied to all elements of the group (i.e. no matter what are the two oringinal messages as long as they belong to the initial group set). This properties defines what we know as **partial homomorphic encryption**.

## Fully homomorphic encription

Ideally we would like to define encryption functions such that they do not only work in one operation (say sum or product) but with the two at the same time. This is what the community coins as fully **homomorphic encryption**. During many years there was a quest to find a fully homomorphic encryption scheme and finally [Craig Gentry](https://en.wikipedia.org/wiki/Craig_Gentry_(computer_scientist)) published his PhD thesis in 2009 with the first fully homomorphic encryption scheme based on [algebraic lattices](https://en.wikipedia.org/wiki/Lattice-based_cryptography).

In [199]:
# TODO: Explain lattices and learning with errrors